In [27]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math as math
import torch
import test_fn

In [28]:
import importlib
importlib.reload(test_fn)

<module 'test_fn' from 'C:\\Users\\Andrew\\OneDrive\\Documents\\HesBO\\baseline\\test_fn.py'>

In [29]:
torch.set_default_dtype(torch.float64)

# Algorithm 1: Shift or twist subspace to best performing points

In [30]:
importlib.reload(test_fn)
fn = test_fn.Ackley()

low_dim = 2
fn_dim = 200

x = torch.rand((100,low_dim))
A = torch.rand((low_dim, fn_dim))
X = x@A
y = fn.evaluate(X)

# Find best performing points (Alt: Find best performing and nearest neighbors)
best_performers_indices = torch.topk(y, 1, largest = False).indices
best_performers = X[best_performers_indices]

# Update
A = best_performers

In [31]:
import torch
from botorch.models import SingleTaskGP
from botorch.fit import fit_gpytorch_model
from gpytorch.mlls import ExactMarginalLogLikelihood

In [32]:
x.shape

torch.Size([100, 2])

In [33]:
import torch
from botorch.models import SingleTaskGP
from botorch.fit import fit_gpytorch_model
from botorch.utils import standardize
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.acquisition import UpperConfidenceBound
from botorch.optim import optimize_acqf

In [34]:
def learn_subspace_best_performers(fn, low_dim, fn_dim, 
                                   num_best = 1, num_neighbors = 0,
                                   twist_subspace = True,
                                   twist_step_size = 1,
                                   move_subspace = False,
                                   move_step_size = 1,
                                   restarts = 0,
                                   samples = 200, max_iter = 1000, 
                                   initial_offset = 2, trace = False):
    
    best_found = 1e10
    A = torch.randn((low_dim, fn_dim))
    B = torch.randn(fn_dim) + initial_offset
    first_run = True
    
#     x = (torch.rand((samples,low_dim)) * 2) - 1
#     X = x@A + B
#     y = fn.evaluate(X)
    
    for i in range(max_iter):
        # Evaluate
        x = (torch.rand((samples,low_dim)) * 2) - 1
        X = x@A + B
        y = fn.evaluate(X)
        y_new = (y - torch.mean(y))/(torch.std(y))
        
        # Bayesian optimization step
        gp = SingleTaskGP(x, y_new.reshape(-1,1) * -1)
        from IPython.core.debugger import set_trace
#         set_trace()
        mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
        fit_gpytorch_model(mll)
        UCB = UpperConfidenceBound(gp, beta=0.1)
        bounds = torch.stack([torch.zeros(low_dim) - 5, torch.zeros(low_dim) + 5])
        candidate, acq_value = optimize_acqf(
            UCB, bounds=bounds, q=1, num_restarts=1, raw_samples=20,
        )
        
        # TODO: actually see if candidate is the best
        # TODO: play with the num_restarts and raw_samples
        
        shift_point = candidate@A + B  

        # Update
        if twist_subspace:
            A = torch.concat([shift_point * twist_step_size, torch.randn((low_dim - num_best, fn_dim))], dim = 0)
        else:
            # Else, reinitialize the transition matrix
            A = torch.randn((low_dim, fn_dim))
            
        if move_subspace:
            if trace:
                print(shift_point.shape)
            B = torch.mean(shift_point * move_step_size, dim = 0)

        # Iterate best found (searching for minimum)
        if min(y) < best_found:
            best_point = X[best_performers_indices[0]]
            best_found = min(y)

        if i%int(max_iter/10) == 0:
            print(f"Step {i} current min/best min found y is:{min(y):.2f}, {best_found:.2f}")
            
#     print(f"Best point found at {best_point}")

In [ ]:
learn_subspace_best_performers(fn, 4, 200, 
                               num_best = 1, num_neighbors = 0,
                               move_subspace = True,
                               move_step_size = 1,
                               restarts = 0)

Step 0 current min/best min found y is:8.95, 8.95
Step 100 current min/best min found y is:2.60, 0.55
Step 200 current min/best min found y is:3.44, 0.55
Step 300 current min/best min found y is:1.87, 0.55
Step 400 current min/best min found y is:4.60, 0.55


In [25]:
%debug

> c:\users\andrew\anaconda3\lib\site-packages\gpytorch\utils\cholesky.py(45)_psd_safe_cholesky()
     43         if not torch.any(info):
     44             return L
---> 45     raise NotPSDError(f"Matrix not positive definite after repeatedly adding jitter up to {jitter_new:.1e}.")
     46 
     47 

ipdb> up
> c:\users\andrew\anaconda3\lib\site-packages\gpytorch\utils\cholesky.py(63)psd_safe_cholesky()
     61             Number of attempts (with successively increasing jitter) to make before raising an error.
     62     """
---> 63     L = _psd_safe_cholesky(A, out=out, jitter=jitter, max_tries=max_tries)
     64     if upper:
     65         if out is not None:

ipdb> up
> c:\users\andrew\anaconda3\lib\site-packages\gpytorch\lazy\lazy_tensor.py(435)_cholesky()
    433 
    434         # contiguous call is necessary here
--> 435         cholesky = psd_safe_cholesky(evaluated_mat, upper=upper).contiguous()
    436         return TriangularLazyTensor(cholesky, upper=upper)
    437 



ipdb> up
> c:\users\andrew\anaconda3\lib\site-packages\gpytorch\module.py(30)__call__()
     28 
     29     def __call__(self, *inputs, **kwargs):
---> 30         outputs = self.forward(*inputs, **kwargs)
     31         if isinstance(outputs, list):
     32             return [_validate_module_outputs(output) for output in outputs]

ipdb> up
> c:\users\andrew\anaconda3\lib\site-packages\botorch\optim\utils.py(214)_scipy_objective_and_grad()
    212         output = mll.model(*train_inputs)
    213         args = [output, train_targets] + _get_extra_mll_args(mll)
--> 214         loss = -mll(*args).sum()
    215     except RuntimeError as e:
    216         if isinstance(e, NotPSDError):

ipdb> up
> c:\users\andrew\anaconda3\lib\site-packages\botorch\optim\utils.py(217)_scipy_objective_and_grad()
    215     except RuntimeError as e:
    216         if isinstance(e, NotPSDError):
--> 217             raise e
    218         if isinstance(e, NanError) or "singular" in e.args[0]:
    219 

ipdb> up
> c:\users\andrew\anaconda3\lib\site-packages\scipy\optimize\lbfgsb.py(360)_minimize_lbfgsb()
    358             # until the completion of the current minimization iteration.
    359             # Overwrite f and g:
--> 360             f, g = func_and_grad(x)
    361         elif task_str.startswith(b'NEW_X'):
    362             # new iteration

ipdb> up
> c:\users\andrew\anaconda3\lib\site-packages\scipy\optimize\_minimize.py(617)minimize()
    615                                   **options)
    616     elif meth == 'l-bfgs-b':
--> 617         return _minimize_lbfgsb(fun, x0, args, jac, bounds,
    618                                 callback=callback, **options)
    619     elif meth == 'tnc':

ipdb> up
> c:\users\andrew\anaconda3\lib\site-packages\botorch\optim\fit.py(239)fit_gpytorch_scipy()
    237 
    238     with gpt_settings.fast_computations(log_prob=approx_mll):
--> 239         res = minimize(
    240             scipy_objective,
    241             x0,

ipdb> up

ipdb> y_new
tensor([-3.8521,  0.9185, -0.9530,  0.7065, -1.1357,  1.9310,  1.1761,  0.3539,
        -1.3691, -1.2063, -0.2259, -0.6405, -1.5632, -0.9067, -0.1836,  1.1327,
        -1.2231, -1.0533,  0.7601, -1.4409, -0.1390,  0.8731,  0.1905,  0.3329,
         0.5756, -2.1139,  0.1074,  0.3498,  0.2334, -0.8409, -2.5937, -0.8723,
        -0.3060, -0.7164,  1.5655,  0.6873,  0.6176, -1.1180,  0.4806,  0.1882,
        -0.3300, -0.1218,  1.0240, -0.6667,  1.3192, -0.1501,  0.4489,  0.6029,
         0.3737,  0.4757,  1.3467, -0.0870,  0.9169,  1.1687,  0.5819,  0.2150,
        -0.4759, -0.4051, -0.1908,  0.1512,  1.1625,  0.3019,  0.7511, -0.5472,
         0.1871, -0.4203,  1.2520,  0.5901, -0.7165,  1.4784,  0.0786, -1.6264,
         0.4052,  1.0395, -0.8651,  0.4845, -0.0212, -1.2763,  0.2949, -0.2028,
        -0.2921,  0.9423,  0.9528,  0.8809,  0.4632,  0.3310,  0.5777,  1.5607,
        -2.4374, -0.1999,  0.1482,  0.5194,  1.6799, -0.3388,  0.7220,  0.2393,
        -0.9852, -0.7038, -0

In [102]:
best_found = 1e10
A = torch.randn((low_dim, fn_dim))
B = torch.randn(fn_dim) + 5
first_run = True            
#     print(f"Best point found at {best_point}")

In [103]:
i = 0
num_best = 1
num_neighbors = 0
move_subspace = True
move_step_size = 1
restarts = 0
samples = 200
twist_subspace = False
trace = False
max_iter = 200

In [113]:
# Evaluate
x = (torch.rand((samples,low_dim)) * 2) - 1
X = x@A + B
y = fn.evaluate(X)

# Bayesian optimization step
gp = SingleTaskGP(x, y.reshape(-1,1))
mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
fit_gpytorch_model(mll)
UCB = UpperConfidenceBound(gp, beta=0.1)
bounds = torch.stack([torch.zeros(low_dim) - 1, torch.zeros(low_dim) + 1])
candidate, acq_value = optimize_acqf(
    UCB, bounds=bounds, q=1, num_restarts=5, raw_samples=20,
)

shift_point = candidate@A + B  

# Update
if twist_subspace:
    A = torch.concat([shift_point * twist_step_size, torch.randn((low_dim - num_best, fn_dim))], dim = 0)
else:
    # Else, reinitialize the transition matrix
    A = torch.randn((low_dim, fn_dim))

if move_subspace:
    if trace:
        print(shift_point.shape)
    B = torch.mean(shift_point * move_step_size, dim = 0)

# Iterate best found (searching for minimum)
if min(y) < best_found:
    best_point = X[best_performers_indices[0]]
    best_found = min(y)

if i%int(max_iter/10) == 0:
    print(f"Step {i} current min/best min found y is:{min(y):.2f}, {best_found:.2f}")

Step 0 current min/best min found y is:14.89, 14.49


In [114]:
candidate

tensor([[-0.7324,  1.0000]])

In [115]:
acq_value

tensor(15.3196)

In [116]:
fn.evaluate(candidate@A + B)

tensor([15.2920])

In [12]:
learn_subspace_best_performers(fn, 4, 200, 
                               num_best = 2, num_neighbors = 0,
                               move_subspace = True,
                               step_size = 0.8,
                               max_iter = 10000,
                               restarts = 0)

Current min/best min found y is:9.00, 9.00
Current min/best min found y is:4.97, 2.26
Current min/best min found y is:8.06, 2.26
Current min/best min found y is:4.24, 2.26
Current min/best min found y is:8.86, 2.26
Current min/best min found y is:3.14, 2.26
Current min/best min found y is:7.42, 1.96
Current min/best min found y is:7.52, 1.96
Current min/best min found y is:8.82, 1.72
Current min/best min found y is:8.18, 1.72


In [76]:
learn_subspace_best_performers(fn, 4, 200, 
                               num_best = 2, num_neighbors = 0,
                               move_subspace = False,
                               step_size = 0.8,
                               max_iter = 10000,
                               restarts = 0)

Current min/best min found y is:5.19, 5.19
Current min/best min found y is:3.29, 2.33
Current min/best min found y is:2.81, 2.33
Current min/best min found y is:3.66, 2.24
Current min/best min found y is:2.87, 2.24
Current min/best min found y is:3.12, 2.24
Current min/best min found y is:3.34, 2.24
Current min/best min found y is:3.62, 2.24
Current min/best min found y is:3.26, 2.24
Current min/best min found y is:3.38, 2.24


In [77]:
learn_subspace_best_performers(fn, 4, 200, 
                               num_best = 2, num_neighbors = 0,
                               move_subspace = True,
                               step_size = 0.8,
                               max_iter = 10,
                               restarts = 0)

Current min/best min found y is:5.35, 5.35
Current min/best min found y is:1.54, 1.54
Current min/best min found y is:4.92, 1.54
Current min/best min found y is:3.02, 1.54
Current min/best min found y is:5.04, 1.54
Current min/best min found y is:3.38, 1.54
Current min/best min found y is:4.96, 1.54
Current min/best min found y is:2.83, 1.54
Current min/best min found y is:4.70, 1.54
Current min/best min found y is:3.10, 1.54


In [19]:
learn_subspace_best_performers(fn, 4, 200, 
                               num_best = 2, num_neighbors = 0,
                               move_subspace = True,
                               step_size = 0.8,
                               max_iter = 5000,
                               restarts = 0,
                               initial_offset = 50)

Current min/best min found y is:21.57, 21.57
Current min/best min found y is:21.57, 3.79
Current min/best min found y is:21.57, 3.32
Current min/best min found y is:21.57, 3.28
Current min/best min found y is:21.57, 3.28
Current min/best min found y is:21.57, 3.28
Current min/best min found y is:21.57, 3.28
Current min/best min found y is:21.57, 3.28
Current min/best min found y is:21.57, 2.80
Current min/best min found y is:21.57, 2.80


In [20]:
learn_subspace_best_performers(fn, 4, 200, 
                               num_best = 2, num_neighbors = 0,
                               move_subspace = False,
                               step_size = 0.8,
                               max_iter = 5000,
                               restarts = 0,
                               initial_offset = 50)

Current min/best min found y is:21.57, 21.57
Current min/best min found y is:3.76, 2.61
Current min/best min found y is:3.38, 2.61
Current min/best min found y is:3.50, 2.44
Current min/best min found y is:3.86, 2.44
Current min/best min found y is:3.23, 2.44
Current min/best min found y is:3.53, 2.44
Current min/best min found y is:3.56, 2.39
Current min/best min found y is:3.18, 2.39
Current min/best min found y is:3.21, 2.39
